---
<center><h1>Spam Mail Prediction</h1></center>
<center><h3>Part of 30 Days 30 ML Projects Challenge</h3></center>

---

## 1) Understanding Problem Statement
---

In the age of digital communication, spam emails have become a significant nuisance, cluttering inboxes and potentially causing harm. This project addresses the issue of spam email detection through the lens of machine learning and data science.

The problem at hand falls into the category of **Binary Classification Machine Learning Problem**. The central objective is **to create a predictive model capable of distinguishing between spam and legitimate emails**. This model will rely on various email attributes and content features to make informed predictions, ultimately helping users filter out unwanted and potentially harmful messages. Moreover, It also involves use of **Natural Language Processing (NLP)** techniques for handling textual data.

## 2) Understanding Data
---

The project uses **Mail Data** which contains several variables (independent variables) and the outcome variable or dependent variable.

## 3) Getting System Ready
---
Importing required libraries


In [3]:
import numpy as np
import pandas as pd

# for text data preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# for model buidling
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

### Downloading stop words for text preprocessing

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## 4) Data Eyeballing
---

### Laoding Data

In [6]:
mail_data = pd.read_csv('Datasets/Day17_Mail_Data.csv') 

In [7]:
mail_data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
print('The size of Dataframe is: ', mail_data.shape)
print('-'*100)
print('The Column Name, Record Count and Data Types are as follows: ')
mail_data.info()
print('-'*100)

The size of Dataframe is:  (5572, 2)
----------------------------------------------------------------------------------------------------
The Column Name, Record Count and Data Types are as follows: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
----------------------------------------------------------------------------------------------------


In [9]:
# Defining numerical & categorical columns
numeric_features = [feature for feature in mail_data.columns if mail_data[feature].dtype != 'O']
categorical_features = [feature for feature in mail_data.columns if mail_data[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 0 numerical features : []

We have 2 categorical features : ['Category', 'Message']


In [10]:
print('Missing Value Presence in different columns of DataFrame are as follows : ')
print('-'*100)
total=mail_data.isnull().sum().sort_values(ascending=False)
percent=(mail_data.isnull().sum()/mail_data.isnull().count()*100).sort_values(ascending=False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

Missing Value Presence in different columns of DataFrame are as follows : 
----------------------------------------------------------------------------------------------------


,Total,Percent
Category,0,0.0
Message,0,0.0


In [11]:
print('Summary Statistics of numerical features for DataFrame are as follows:')
print('-'*100)
mail_data.describe(include='object')

Summary Statistics of numerical features for DataFrame are as follows:
----------------------------------------------------------------------------------------------------


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


## 5) Data Cleaning and Preprocessing
---

### Replace the null values with a null string (Only for demonstration, here it is not neede)

In [12]:
mail_data = mail_data.where((pd.notnull(mail_data)),'')

In [13]:
mail_data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


### Label Encoding

#### label spam mail as 0;  ham mail as 1;

In [14]:
mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

In [15]:
mail_data

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will ü b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...


In [16]:
mail_data['Category'].value_counts()

Category
1    4825
0     747
Name: count, dtype: int64

### Stemming

In [17]:
porter_stemmer = PorterStemmer()

In [18]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [porter_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [19]:
mail_data['Message'] = mail_data['Message'].apply(stemming)

In [20]:
mail_data['Message']

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri wkli comp win fa cup final tkt st m...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    nd time tri contact u u pound prize claim easi...
5568                                b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: Message, Length: 5572, dtype: object

## 6) Model Building
---

### Creating Feature Matrix (Independent Variables) & Target Variable (Dependent Variable)

In [42]:
# separating the data and labels
X = mail_data['Message'] # Feature matrix
y = mail_data['Category'] # Target variable

In [43]:
X

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri wkli comp win fa cup final tkt st m...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    nd time tri contact u u pound prize claim easi...
5568                                b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: Message, Length: 5572, dtype: object

In [44]:
y

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object

In [53]:
# convert Y_train and Y_test values as integers

y = y.astype('int')

### Feature Extraction

#### Transform the text data to feature vectors that can be used as input to the Logistic regression

In [54]:
vectorizer = TfidfVectorizer()

In [55]:
vectorizer.fit(X)

X = vectorizer.transform(X)

AttributeError: lower not found

In [ ]:
X

In [ ]:
print(X)

### Train-Test Split

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=45)

In [57]:
print(X.shape, X_train.shape, X_test.shape)

(5572, 6296) (4457, 6296) (1115, 6296)


In [58]:
print(y.shape, y_train.shape, y_test.shape)

(5572,) (4457,) (1115,)


### Model Comparison : Training & Evaluation

In [59]:
models = [LogisticRegression, SVC, DecisionTreeClassifier, RandomForestClassifier]
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for model in models:
    classifier = model().fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

In [60]:
classification_metrics_df = pd.DataFrame({
    "Model": ["Logistic Regression", "SVM", "Decision Tree", "Random Forest"],
    "Accuracy": accuracy_scores,
    "Precision": precision_scores,
    "Recall": recall_scores,
    "F1 Score": f1_scores
})

classification_metrics_df.set_index('Model', inplace=True)
classification_metrics_df

,Accuracy,Precision,Recall,F1 Score
Model,,,,
Logistic Regression,0.953363,0.949803,0.998965,0.973764
SVM,0.972197,0.968907,1.000000,0.984208
Decision Tree,0.963229,0.975334,0.982402,0.978855
Random Forest,0.974888,0.971831,1.000000,0.985714


### Inference

In the context of spam mail prediction, the performance metrics of four different models—Logistic Regression, SVM, Decision Tree, and Random Forest—have been evaluated. 

- Overall, all models exhibit high accuracy, with SVM and Random Forest leading the way, achieving accuracy scores of approximately 97.22% and 97.49%, respectively. This suggests that these models are effective at correctly classifying emails as spam or legitimate.

- When considering precision, recall, and F1 Score, the SVM and Random Forest models again excel, with precision scores exceeding 96.89% and recall scores reaching 100%, indicating minimal false positives and false negatives. Decision Tree also performs well in terms of precision and recall.

In summary, SVM and Random Forest models stand out as strong contenders for spam mail prediction, with high accuracy, precision, and recall scores. However, the choice of the best model may also depend on specific requirements such as computational efficiency and interpretability.